In [1]:
import os
from pathlib import Path

import torch as t
import pytorch_lightning as pl
import torchvision as tv

from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization


In [2]:
pl.seed_everything(7)

Global seed set to 7


7

In [13]:
# DATAROOT = Path.home() / "cifar10"
DATAROOT = Path.home() / "mldata" / "cifar10"
N_WORKERS = int(os.cpu_count() / 2)

# Hyperparams
BATCH_SIZE = 256

In [14]:
norms = cifar10_normalization()
print(type(norms), norms)

<class 'torchvision.transforms.transforms.Normalize'> Normalize(mean=[0.4913725490196078, 0.4823529411764706, 0.4466666666666667], std=[0.24705882352941178, 0.24352941176470588, 0.2615686274509804])


In [15]:
train_xforms = tv.transforms.Compose([
    tv.transforms.RandomCrop(32, padding=4),
    tv.transforms.RandomHorizontalFlip(),
    tv.transforms.ToTensor(),
    cifar10_normalization()
])

In [16]:
test_xforms = tv.transforms.Compose([
    tv.transforms.ToTensor(),
    cifar10_normalization()
])

In [17]:
data = CIFAR10DataModule(
    data_dir=DATAROOT,
    batch_size=BATCH_SIZE,
    num_workers=N_WORKERS,
    train_transforms=train_xforms,
    test_transforms=test_xforms,
    val_transforms=test_xforms
)

/home/avilay/miniconda3/envs/pt/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:73: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/home/avilay/miniconda3/envs/pt/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:77: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/home/avilay/miniconda3/envs/pt/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:81: LightningDeprecationWarning: DataModule property `test_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(


In [18]:
data.prepare_data()

Files already downloaded and verified
Files already downloaded and verified


In [19]:
data.setup("fit")

/home/avilay/miniconda3/envs/pt/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:114: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/home/avilay/miniconda3/envs/pt/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:133: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(


In [24]:
traindl = data.train_dataloader()

In [25]:
lens = []
for batch in traindl:
    lens.append(batch[1].shape[0])
print(len(lens), sum(lens))

157 40000


In [25]:
batch = next(iter(traindl))

In [26]:
type(batch)

list

In [27]:
len(batch)

2

In [29]:
print(type(batch[0]), type(batch[1]))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [30]:
print(batch[0].shape, batch[1].shape)

torch.Size([64, 3, 32, 32]) torch.Size([64])


## Resnet Model

Resnet model was trained with the image net dataset and so has the last affine layer has a 1000 units. I can override this by specifying the `num_classes` parameter.

Further, the first convolution layer in the default resnet model has a kernel size of 7x7, stride length of 2x2, and padding of 3x2, which is suitable for 224x224 imagenet photos. Cifar10 photos are 32x32 in size, so we have to reduce the kernel size, stride length, and padding to 3x3, 1x1, and 1x1 respectively.

Further the default resenet model has a maxpool layer following the conv2d/relu layers. For some reason in the lightning tutorial this is replaced with an identity module, which I am guessing is a no-op layer.

In [14]:
model = tv.models.resnet18(pretrained=False)

In [15]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [16]:
model = tv.models.resnet18(pretrained=False, num_classes=10)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [17]:
model.conv1 = t.nn.Conv2d(3, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
model.maxpool = t.nn.Identity()

In [18]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1